In [38]:
from collections import Counter
from itertools import product
import math
import io
import os
import time

In [39]:
#PREPROCESSING
def get_ngrams(path, n):
  result = []
  with open(path) as input:
    for line in input:
      if(len(line)<3 or len(line)>16):
        continue
      line = "0" + line
      for i in range(len(line) - (n-1)):
        result.append((line[i:i+n]))
  return result

get_ngrams("tester.txt", 3)


def post_process(input):
  result = []
  for value in input:
    result.append(value[1:-1])
  return result

In [65]:
#HELPER FUNCTIONS

#turns a 2d matrix into an adjacency list of indeces
def board_to_adjlist(board):
    result = dict()
    for i in range(len(board)):
        for j in range(len(board[0])):
            idx = (i,j)
            neighbors = []
            for x in range(-1,2):
                for y in range(-1,2):
                    if(not(x == 0 and y ==0)):
                        if(x + i >=0 and y + j >=0 and x + i < len(board) and y + j < len(board[0])):
                            neighbors.append((i+x,j+y))
            result[idx] = neighbors
    return result

#turns a 2d matrix into a list of characters
def board_to_list(board):
  result = []
  for row in board:
    result = result + row
  result.sort()
  return result

def board_to_dict(board):
  result = dict()
  for i in range(len(board)):
    for j in range(len(board[0])):
      if(board[i][j] not in result):
        result[board[i][j]] = [(i,j)]
      else:
        result[board[i][j]].append((i,j))
  return result

def exists(word, chars):
  #REQUIRES: char is sorted
  word = list(word)
  word.sort()
  curr = 0
  i = 0
  length = len(chars)
  goal = len(word)
  while(i < length):
    if(curr == goal):
      return True
    if(word[curr] == chars[i]):
      curr +=1
    elif(word[curr] < chars[i]):
      return False
    i +=1
  return curr == goal 

def validate(dictionary, result):
  res = []
  for word in result:
    num = hash(word) % dictionary.size
    if (num in dictionary.book) and (word in dictionary.book[num]):
      res.append(word)
  return res 

# Building a Literal Hash Dictionary

In [41]:
class Dictionary():
  def __init__(self, path, size):
    self.size = size
    self.book = self.build(path, size)
    self.listed = self.to_list(path)
  
  def build(self, path, size):
    result = dict()
    with open(path) as input:
      for line in input:
        line = line[:-1]
        num = hash(line) % size
        if num in result:
          result[num].append(line)
        else:
          result[num] = [line]
    return result
  
  def to_list(self, path):
    result = []
    with open(path) as input:
      for line in input:
        result.append(line[:-1])
    return result

  def get_word(self, word):
    num = hash(word) % self.size
    return (num in self.book and word in self.book[num])
  






# Building Our N-Gram Language Model

In [42]:
class LanguageModel():
    def __init__(self, n, train_data, alpha=1):
        """
        Language model class.
        
        Args
        ____
        n: int
            n-gram order
        train_data: List[List]
            list of words in the dictionary that have not been preprocessed yet
        alpha: float
            Smoothing parameter
        
        Other required parameters:
            self.vocab: vocabulary dict with counts
            self.model: n-gram language model, i.e., n-gram dict with probabilties
            self.n_grams_counts: Frequency count for each of the n-grams present in the training data
            self.prefix_counts: Frequency count of all the corresponding n-1 grams present in the training data
        """
        self.n = n
        self.train_data = train_data
        self.tokens = self.train_data
        self.n_grams_counts = None
        self.prefix_counts = None
        self.vocab  = Counter(self.tokens)
        self.alpha = alpha
        self.model = self.build()

    def get_smooth_probabilities(self,n_gram):
      vocab_size = len(self.vocab)
      ngram_sum = self.n_grams_counts[n_gram]

      prefix_sum = len(self.tokens)
      if(self.n>1):
        prefix_sum = self.prefix_counts[n_gram[0:self.n-1]]
        
        

      return (ngram_sum + self.alpha)/(prefix_sum + self.alpha*vocab_size)


      #Returns the smoothed probability of a single ngram, using Laplace Smoothing. Remember to handle the special case of n=1
      #Use the class variables we defined in the build function. It is suggested to implement the build function before this one.

    
    #TODO:
    def build(self):
        
        #Returns a n-gram (could be a unigram) dict with n-gram tuples as keys and probabilities as values. 
        #It could be a unigram model as well
        ngrams = get_ngrams(self.train_data,n=self.n)
        prefix_ngrams = get_ngrams(self.train_data,n = self.n-1)

        self.n_grams_counts = Counter(ngrams)
        self.prefix_counts = Counter(prefix_ngrams)

        prob = dict()
        for n_gram in self.n_grams_counts:
          prob[n_gram] = self.get_smooth_probabilities(n_gram)

        return prob

# Albert: N-Gram Language Model-Based Solver

In [90]:
class Albert():
  def __init__(self, model,n):
    # I want to pass in a naive bayes n-gram model here.
    #the probability "p" is the parameter we want to train
    self.model = model
    self.p = 0.03
    self.n = n
  

  def bfs(self, tile, adjlist, board, path, result, visited):
    #INPUT
    #tile : the root tile that we are starting from
    #adjlist : adjacency list of our graph
    #board : 2d matrix of our graph so accessing is easier
    #path : the current accumulated letters for our words
    #pathprob : should we add the log probability of the current path? will do it on second try
    #result : the resulting set from the past
    #visited : the tiles that we have already visited

    #OUTPUT
    #result : the resulting set after conducting bfs

    #check if current path is a word
    finished = path[-(self.n-1):] + "\n"
    if(len(path) > 3 and finished in self.model and self.model[finished] >= self.p):
      result.add(path + "\n") 

    #check if extensions are a word
    #for each neighbor
    for neighbor in adjlist[tile]:
        #if we haven't visited it yet, we do smth
        #so if we've visited all the neighbors we just move on
        if neighbor not in visited:
            next = str(board[neighbor[0]][neighbor[1]])
            candidate = path[-(self.n-1):] + next
            #check probabilities
            if(candidate in self.model and self.model[candidate] >= self.p):
              new = path + next
              temp = visited.copy()
              temp.append(neighbor)
              result = result | self.bfs(neighbor, adjlist, board, new, result, temp)
    return result 


  def solve(self, board):
      #INPUT
      # board: a 2-d char array of n x n dimensions
      # cutoff: the probability 0 < cutoff < 1, such that if the probability a character follows is lower than this cutoff we do not pursue this path

      #OUTPUT
      # result: a set of words found from the board

      result = set()

      #first, we want to make an adjacency list
      adjlist = board_to_adjlist(board)
      for tile in adjlist:
        visited = [tile]
        path = "0" + str(board[tile[0]][tile[1]])
        found = self.bfs(tile, adjlist, board,path , set(), visited)
        result = result | found


      
      return post_process(result)
  
  def train(self, boards, solutions):
    #essentially, what we want to do here is solve each board
    #its better to make our probability initially really small so that
    #runtime isn't dogshit (it will take eons to run if we go through every single possible path)
    #then our loss function will be p = p * log (wP/wR) (thinking about adding here, see which one does better), 
    # where wP is the weighted precision and wR is the weighted recall (this is based on wordhunt scores, eg 800 for 5 letter words)
    #we keep going until we go through all the boards and solutions
    #afterwards, print our final p-value to see where it ended up
    return
  

  def evaluate(self, answer, reference):
    #this is where we calculate the loss function stuff
    return

# Isaac: Dictionary-Based Solver

In [44]:
class Isaac():
  def __init__(self, dictionary):
    self.dictionary = dictionary.listed
  

  def dfs(self, word, tile, adjlist, visited, path, board):
    if(path == len(word)):
      return True
    
    for neighbor in adjlist[tile]:
      if neighbor not in visited:
        if board[neighbor[0]][neighbor[1]] == word[path]:
          temp = visited.copy()
          temp.append(neighbor)
          if(self.dfs(word,neighbor, adjlist, temp, path + 1, board)):
            return True

    return False



  def solve(self, board):
    result = []
    adjlist = board_to_adjlist(board)
    lst = board_to_list(board)
    ref = board_to_dict(board)
    #im going to make a dictionary for easy access

    for word in self.dictionary:
      if len(word) > 2 and len(word) <= 16 and exists(word, lst):
        for start in ref[word[0]]:
          if(self.dfs(word, start, adjlist, [start], 1, board)):
            result.append(word)
            break
    return result


# Testing Begins!

In [79]:

tester = [['H','A','M','R'], ['U','Y','E','A'], ['R','D','N','N'], ['L','L','E','K']]

In [49]:
dictionary = Dictionary("dictionary.txt",10000)

In [ ]:
lm = LanguageModel(3, "dictionary.txt")
albert = Albert(lm.model, 3)

In [106]:
tic = time.perf_counter()
result = albert.solve(tester)
result = validate(dictionary, result)
toc = time.perf_counter()
print(toc - tic)
result = sorted(result, key = len, reverse = True)
print(result)
print(len(result))

0.13382851499954995
['REMANNED', 'MANNER', 'LENDER', 'ENDEAR', 'MANNED', 'YEANED', 'REMAND', 'AMEND', 'MANED', 'ENDER', 'DERMA', 'ARENE', 'REMAN', 'DENAR', 'RUDER', 'HYMEN', 'RAMEN', 'YAMEN', 'NAMED', 'ARMED', 'NAMER', 'LEND', 'YEAR', 'RAND', 'NAME', 'REDE', 'YEAN', 'MEND', 'DELL', 'MANE', 'DERM', 'DEAN', 'MARE', 'KNAR', 'DENE', 'REND', 'AMEN', 'HAME', 'REAM', 'RUDE', 'DEAR', 'KEN', 'DEN', 'MAN', 'ARM', 'DEL', 'RED', 'HAM', 'YEN', 'MED', 'YAM', 'AND', 'DRY', 'LED', 'EAR', 'NAM', 'RAM', 'MEN', 'ANE', 'END', 'NAN', 'ELL', 'ARE', 'RAN', 'AMA', 'MAR']
66
41


In [ ]:
isaac = Isaac(dictionary)

In [105]:
tic = time.perf_counter()
result = isaac.solve(tester)
toc = time.perf_counter()
result = sorted(result, key = len, reverse = True)
print(toc- tic)
print(result)
print(len(result))
long = [item for item in result if len(item)>3]
print(len(long))

0.4627254580009321
['REMANNED', 'ENDEAR', 'HURDLE', 'HURLED', 'HYAENA', 'KENNED', 'LENDER', 'MANNED', 'MANNER', 'RANKED', 'REDENY', 'REMAND', 'YEANED', 'YENNED', 'AMEND', 'ARENE', 'ARMED', 'DENAR', 'DERMA', 'DRYER', 'DYNEL', 'EDEMA', 'ELDER', 'EMYDE', 'ENDER', 'ENEMA', 'ENEMY', 'HAYED', 'HAYER', 'HYENA', 'HYMEN', 'KNELL', 'MANED', 'MAYED', 'MEANY', 'NAMED', 'NAMER', 'RAMEN', 'RANDY', 'REDRY', 'REMAN', 'RUDER', 'YAMEN', 'AMAH', 'AMEN', 'AREA', 'ARMY', 'DEAN', 'DEAR', 'DELL', 'DEMY', 'DENE', 'DENY', 'DERM', 'DYER', 'DYNE', 'EMYD', 'EYNE', 'HAED', 'HAEM', 'HAEN', 'HAME', 'HURL', 'KNAR', 'LEND', 'MANE', 'MANY', 'MARE', 'MAUD', 'MEAN', 'MEND', 'MYNA', 'NAME', 'NEAR', 'NEMA', 'NENE', 'RAND', 'RANK', 'REAM', 'REDE', 'REND', 'RUDE', 'RYND', 'YAUD', 'YEAH', 'YEAN', 'YEAR', 'AMA', 'AND', 'ANE', 'ANY', 'ARE', 'ARM', 'AYE', 'DEL', 'DEN', 'DEY', 'DRY', 'DUH', 'DYE', 'EAR', 'EAU', 'ELD', 'ELL', 'END', 'ERA', 'HAE', 'HAM', 'HAY', 'KEN', 'LED', 'LEK', 'MAE', 'MAN', 'MAR', 'MAY', 'MED', 'MEN', 'NAE', '

In [103]:
lm.model["AYE"]

0.1490933512424446